# Reuse of phasepy functions

The purpose of this notebook is to show how the necessary methods and attributes for an object to be used alongside phasepy functions for phase equilibria and square gradient theory.

- [Pure Fluid](#pure-fluids)
- [Fluid Mixtures](#mixtures)

In [1]:
import numpy as np
from phasepy import component, mixture, preos

<a id='pure-fluids'></a>


## 1. Pure Fluids EoS Methods and Attributes

You can reuse the ```sgt_pure``` function to study the interfacial behavior of a pure fluid. For this function you must supply an object which must include the following methods:

- ```eos.sgt_adim```: Method that evaluates dimensionless factors for temperature, pressure, density, tension, and distance for interfacial properties computations with SGT.
- ```eos.muad```: Method that computes the dimensionless chemical potential at given density and temperature.
- ```eos.a0ad```: Method that computes the dimensionless Helmholtz density energy at given density and temperature.
- ```eos.dOm```: Method that computes the dimensionless Thermodynamic Grand potential at given density and temperature.
- ```eos.ci``` :  Computes influence parameters for SGT.

In [2]:
water =  component(name = 'Water', Tc = 647.13, Pc = 220.55, Zc = 0.229, Vc = 55.948, w = 0.344861,
                   ksv = [ 0.87185176, -0.06621339], cii = [2.06553362e-26, 2.64204784e-23, 4.10320513e-21])

eosp = preos(water)

You must be able to compute the VLE of the fluid. In phasepy's cubic eos you can do that with the ```eos.psat``` method.

In [3]:
Tsat = 298.15 # K
Psat, vl, vv = eosp.psat(Tsat)
rhol = 1. / vl
rhov = 1. / vv
# Saturation pressure (bar), liquid density (mol/cm3), vapor density (mol/cm3)
Psat, rhol, rhov

(array([0.02665047]), array([0.04708998]), array([1.07556519e-06]))

---

####  ```eos.sgt_adim``` method:
- Parameters: `T` (Temperature)
- Returns: `Tfactor`, `Pfactor`, `rhofactor`, `tenfactor`, `zfactor`

For pure fluids, Square Gradient Theory (SGT) states that the interfacial tension is computed as follows. 

$$ \gamma = \sqrt{2c_{ii}} \int_{\rho_v}^{\rho_l} \underbrace{\left[a_0 - \rho \mu^0 + P^0 \right]}_{\Delta \Omega} d\rho \rightarrow \gamma^* = \sqrt{2} \int_{\rho_v^*}^{\rho_l^*} \underbrace{\left[a_0^* - \rho \mu^{0*} + P^{0*} \right]}_{\Delta \Omega^*} d\rho^*$$

Where $\gamma$ is the interfacial tension, $c_{ii}$ is the influence parameter, $\rho$ is the fluid density, $a_0$ is the Helmholtz density energy, $\mu$ is the chemical potential, $P$ is the pressure, and $\Delta \Omega$ is the Thermodynamic Grand Potential Difference. Superscript $0$ refers to properties evaluated at equilibrium, and superscript $*$ refers to the dimensionless properties. Subscript $l$ and $v$, refers to liquid and vapor phase, respectively.

The dimensionless quantities are obtaines as shown below:

- $T^* = Tfactor \cdot T$
- $P^* = Pfactor \cdot P$
- $\rho^* = rhofactor \cdot \rho$
- $ z^* = zfactor \cdot z$
- $ \gamma = tenfactor \cdot \gamma^*$

The ``eos.sgt_adim``  included in phasepy's cubic eos computes the following factors:

- $Tfactor = R \cdot b / a$
- $Pfactor = b^2 / a$
- $rhofactor = b$
- $tenfactor = 1000*\sqrt{a \cdot c_{ii}}/b^2*(\sqrt{101325/1.01325} \cdot 100^3)$
- $zfactor =  \sqrt{a/c_{ii}*10^5/100^6}*10^{-10}$

Where $a$ and $b$ are the cohesive and covolume from the cubic EoS and $R$ is the ideal gas constant. These conversion factors allow working with dimensionally consistent SGT equations and will compute the interfacial length in Angstrom  ($\rm \mathring A$) and interfacial tension in mN/m.

You can set the factors to 1 if your unit system is consistent but be aware that the influence parameter is included in the SGT equation through the ```tenfactor``` and ```zfactor``` parameters. 


In [4]:
Tfactor, Pfactor, rhofactor, tenfactor, zfactor = eosp.sgt_adim(Tsat)

# The factors are used as follows:
rola = rhol * rhofactor
rova = rhov * rhofactor
Tad = Tsat * Tfactor
Pad = Psat * Pfactor

---

####  ```eos.ci``` method:
- Parameters: `T` (Temperature)
- Returns: `cii` (influence parameter)

In phasepy's cubic EoS objects the ``eos.ci`` method is called by the ``eos.sgt_adim`` method.



In [5]:
# computing influence parameter at given temperature
eosp.ci(Tsat)

1.3816594292666143e-20

---

#### `eos.a0ad` method:
- Parameters: `rhoa` (dimensionless density), `Tad` (dimensionless Temperature)
- Returns: `a0` (dimensionless Helmholtz density energy)


The ```eos.a0ad``` method computes the dimensionless Helmholtz density energy ($a_0$) at given dimensionless density and dimensionless temperature.

In [6]:
eosp.a0ad(rova, Tad)

array([-1.34987859e-05])

---

#### ```eos.muad``` method:
- Parameters: `rhoa` (dimensionless density), `Tad` (dimensionless Temperature)
- Returns: `mu` (dimensionless chemical potential)

The ```eos.muad``` method computes the dimensionless chemical potential ($\mu$) at a given dimensionless density and dimensionless temperature.

In [7]:
# Equilibrium chemical potential
mu0 = eosp.muad(rova, Tad)
mu02 = eosp.muad(rola, Tad)
mu0, mu02

(array([-0.6135512]), array([-0.61355089]))

--- 
#### ```eos.dOm``` method:
- Parameters: `rhoa` (dimensionless density), `Tad` (dimensionless Temperature), `mu0` (dimensionless equilibrium chemical potential), `Pad` (dimensionless equilibrium pressure)
- Returns: `dom` (dimensionless Thermodynamic Grand Potential difference)


The ```eos.dOm``` will compute the Grand Thermodynamic Potential difference ($\Delta \Omega$) at a given dimensionless density, temperature, equilibrium chemical potential, and pressure.

In [8]:
# Thermodynamic Grand Potential difference
rhoa = 10.*rova
eosp.dOm(rhoa, Tad, mu0, Pad)

array([1.36469886e-05])

<a id='mixtures'></a>


## 2. Fluid Mixtures EoS Methods and Attributes

For fluid mixtures you can compute the reuse phase equilibrium and stability functions, such as: ```tpd_min```, ```tpd_minimas```, ```lle_init```, ```flash```, ```bubblePy```, ```bubbleTy```, ```dewTx```, ```dewPx```, ```lle```, ```vlleb```, ```vlle```.

Additionally, you can reuse the implemented functions for Square Gradient for mixtures, such as: ```sgt_mix_beta0```, ```sgt_mix```, ```msgt_mix```.

In order to use these functions, the EoS object must include the following methods and attributes:

**methods:**

- ```eos.temperature_aux```: computes temperature dependent parameters.
-  ```eos.logfugef_aux```: computes effective fugacity coefficients.
- ```eos.dlogfugef_aux```: computes effective fugacity coefficients and it composition derivatives (OPTIONAL).
- ```eos.a0ad_aux```: computes dimensionless Helmholtz density energy.
- ```eos.muad_aux``` : computes dimensionless chemical potential.
- ```eos.dmuad_aux``` : computes dimensionless chemical potential and its composition derivatives (OPTIONAL).
- ```eos.dOm_aux``` : computes dimensionless Thermodynamic Grand Potential Difference.
- ```eos.ci``` :  computes influence parameters matrix for SGT.
- ```eos.sgt_adim``` : computes dimensionless factors for SGT.

**attributes:**

- ```eos.secondorder```: Boolean, ``True`` if ```eos.dlogfugef_aux``` is available.
- ```eos.secondordersgt```: Boolean, ``True`` if ```eos.dmuad_aux``` is available.

In [9]:
water =  component(name = 'Water', Tc = 647.13, Pc = 220.55, Zc = 0.229, Vc = 55.948, w = 0.344861,
                ksv = [ 0.87185176, -0.06621339],
                cii = [2.06553362e-26, 2.64204784e-23, 4.10320513e-21],
                GC = {'H2O':1})

ethanol = component(name = 'Ethanol', Tc = 514.0, Pc = 61.37, Zc = 0.241, Vc = 168.0, w = 0.643558,
                ksv = [1.27092923, 0.0440421 ],
                cii = [ 2.35206942e-24, -1.32498074e-21,  2.31193555e-19],
                GC = {'CH3':1, 'CH2':1, 'OH(P)':1})

mix = mixture(ethanol, water)
# or
mix = ethanol + water
mix.unifac()
eosm = preos(mix, 'mhv_unifac')

---
#### `eos.temperature_aux` method:
- Parameters: `T` (Temperature)
- Returns: `temp_aux` (Tuple with all temperature dependent parameters)

The ```eos.temperature_aux``` will compute all the EoS temperature-dependent parameters. This method returns a tuple which is used in all auxiliary methods ( ```eos.method_aux```).

In [10]:
T = 298.15 # K
temp_aux = eosm.temperature_aux(T)

---
#### `eos.logfugef_aux` method:
- Parameters: `x` (Mole fraction array), `temp_aux` (Tuple with Temperature-dependent parameters), `P` (Pressure), `state` (aggregation state: `'L'` or `'V'`) and `v0` (optionally, to solve volume root)
- Returns: `logfugef` (natural logarithm of the effective fugacity coefficients) and `v` (volume root)

For the phase equilibria and stability functions, you need to provide an expression for the effective fugacity coefficient ($\ln \hat{\phi_i}$).

The natural logarithm of the effective fugacity coefficients is computed with the ```eos.logfugef_aux``` method. This method requires the mole fraction vector, the temp_aux tuple obtained from the ```eos.temperature_aux``` method, the pressure, the aggregation state (```'L'``` or ```'V'```) and the option to include an initial guess to compute the volume of the mixture at the given conditions. 

In [11]:
T = 298.15 # K
P = 1.01325 # bar
temp_aux = eosm.temperature_aux(T)
x = np.array([0.2, 0.8])
logfugef, vlm = eosm.logfugef_aux(x, temp_aux, P, 'L', v0=None)
logfugef, vlm

(array([-1.74849038, -3.55682539]), 29.3806008794218)

---
#### `eos.dlogfugef_aux` method:
- Parameters: `x` (Mole fraction array), `temp_aux` (Tuple with Temperature-dependent parameters), `P` (Pressure), `state` (aggregation state: `'L'` or `'V'`) and `v0` (optionally, to solve volume root)
- Returns: `logfugef` (natural logarithm of the effective fugacity coefficients), `dlogfugef` (natural logarithm of the effective fugacity coefficients composition derivatives) and `v` (volume root)

Optionally, for equilibria computation at fixed temperature and pressure (```flash``` or ```vlle```) you can speed up the computation if you provide the derivatives of the fugacity coefficient ($d\ln \hat{\phi_i} / dn_j)$. 

The method ``eos.dlogfugef_aux`` computes the fugacity coefficient, its composition derivatives, and the volume root. This function requires the mole fraction vector, the temp_aux tuple obtained from the ```eos.temperature_aux``` method, the pressure, the aggregation state (```'L'``` or ```'V'```) and the option to include an initial guess to compute the volume of the mixture at the given conditions. 

You can indicate that this method is available setting the ``eos.secondorder`` attribute to ``True``.

In [12]:
print('eos.secondorder: ', eosm.secondorder)

T = 298.15 # K
P = 1.01325 # bar
temp_aux = eosm.temperature_aux(T)
x = np.array([0.2, 0.8])
logfugef, dlogfug, vlm = eosm.dlogfugef_aux(x, temp_aux, P, 'L', v0=None)
logfugef, dlogfug, vlm 

eos.secondorder:  True


(array([-1.74849038, -3.55682539]),
 array([[-2.26756678,  0.56689169],
        [ 0.56689169, -0.14172292]]),
 29.38060087942181)

---

####  ```eos.sgt_adim``` method:
- Parameters: `T` (Temperature)
- Returns: `Tfactor`, `Pfactor`, `rhofactor`, `tenfactor`, `zfactor`

For mixtures, SGT states that the density profiles must fulfill the following Euler-Lagrange system:

$$ \sum_j c_{ij} \frac{d^2 \rho_j}{dz^2} = \mu_i - \mu_i^0 \quad \rightarrow \quad  \sum_j \frac{c_{ij}}{c_{11}} \frac{d^2 \rho_j^*}{dz^{*2}} = \mu_i^* - \mu_i^{*0} \\
\rho(z \rightarrow -\infty) = \rho^\alpha \quad \rho(z \rightarrow \infty) = \rho^\beta  \quad \rightarrow \quad
\rho(z^* \rightarrow -\infty) = \rho^{*\alpha} \quad \rho(z^* \rightarrow \infty) = \rho^{*\beta} $$ 


Then the dimensionless interfacial tension is computed as follows:

$$ \gamma^* = \int_{0}^{L^*} \sqrt{2 \Delta \Omega^*(\rho^*)  \left( \sum_{i=1}^c \sum_{j=1}^c \frac{c_{ij}}{c_{11}} \frac{d\rho^*_i}{dz^*} \frac{d\rho^*_j}{dz^*} \right) }dz^* $$

The dimensionless quantities are obtaines as shown below:

- $T^* = Tfactor \cdot T$
- $P^* = Pfactor \cdot P$
- $\rho^* = rhofactor \cdot \rho$
- $ z^* = zfactor \cdot z$
- $ \gamma = tenfactor \cdot \gamma^*$

Similarly as for pure fluids, the ```eos.sgt_adim``` computes the factors to work with a dimensionless SGT's equations. 

- $Tfactor = R \cdot b_1 / a_1$
- $Pfactor = b_1^2 / a_1$
- $rhofactor = b_1$
- $tenfactor = 1000*\sqrt{a_1 \cdot c_{11}}/b_1^2*(\sqrt{101325/1.01325} \cdot 100^3)$
- $zfactor =  \sqrt{a_1/c_{11}*10^5/100^6}*10^{-10}$

Where, $a_1$ and $b_1$ are the cohesive and covolume from the cubic EoS of the first component of the mixture. These conversion factors allow working with dimensionally consistent SGT equations and will compute the interfacial length in Angstrom  ($\rm \mathring A$) and interfacial tension in mN/m.

You can set the factors to 1 if your unit system is consistent but be aware that the influence parameter of the first component is used when solving the density profiles and IFT. You need to re-include it through the ```tenfactor``` and ```zfactor``` parameters. 

In [13]:
T = 298.15 # K
Tfactorm, Pfactorm, rhofactorm, tenfactorm, zfactorm = eosm.sgt_adim(T)

---

####  ```eos.ci``` method:
- Parameters: `T` (Temperature)
- Returns: `cij` (influence parameter matrix)


The influence parameter matrix is computed with the ```eos.ci``` method.

In [14]:
T = 298.15 # K
cij = eosm.ci(T)

---
#### `eos.a0ad_aux` method:
- Parameters: `rhoa` (dimentionless density vector), `temp_aux` (Tuple with Temperature-dependent parameters)
- Returns: `a0ad` (dimensionless Helholmtz density energy)

The dimensionless Helholmtz density energy ($a_0$) is computed with the ```eos.a0ad_aux``` method. This method requires the dimensionless density vector and the tuple computed with the ```eos.temperature_aux``` method.

In [15]:
rhoad = x / vlm * rhofactorm
eosm.a0ad_aux(rhoad, temp_aux)

-0.2892854573806896

---
#### `eos.muad_aux` method:
- Parameters: `rhoa` (dimentionless density vector), `temp_aux` (Tuple with Temperature-dependent parameters)
- Returns: `muad` (dimensionless chemical potential array)

The method ```eos.muad_aux``` computes the dimensionless chemical potential ($\mu_i$) at a given density vector and temperature. This method requires the dimensionless density vector and the tuple computed with the ```eos.temperature_aux``` method.

In [16]:
rhoad = x / vlm * rhofactorm

muad = eosm.muad_aux(rhoad, temp_aux)
muad 

array([-0.1370864 , -0.16176095])

---
#### `eos.dmuad_aux` method:
- Parameters: `rhoa` (dimentionless density vector), `temp_aux` (Tuple with Temperature-dependent parameters)
- Returns: `muad` (dimensionless chemical potential array) and `dmuad` (dimensionless chemical potential array composition derivatives)

The method ```eos.dmuad_aux``` computes the chemical potential ($\mu_i$) and its composition derivatives matrix ($d\mu_i/d\rho_j$)  at given density vector and temperature. This method requires the dimensionless density vector and the tuple computed with the ```eos.temperature_aux``` method.

This method is used if the ```eos.secondordersgt``` attribute is set to ```True``` and allows a faster density profile computation.

In [17]:
print('eos.secondordersgt: ', eosm.secondordersgt)
rhoad = x / vlm * rhofactorm

muad, dmuad = eosm.dmuad_aux(rhoad, temp_aux)
muad, dmuad

eos.secondordersgt:  True


(array([-0.1370864 , -0.16176095]),
 array([[8.52913634, 2.87491965],
        [2.87491965, 0.98810607]]))

--- 
#### ```eos.dOm_aux``` method:
- Parameters: `rhoa` (dimentionless density vector), `temp_aux` (Tuple with Temperature-dependent parameters), `mu0` (dimensionless equilibrium chemical potential), `Pad` (dimensionless equilibrium pressure)
- Returns: `dom` (dimensionless Thermodynamic Grand Potential difference)

The Thermodynamic Grand Potential difference ($\Delta \Omega$) is computed with the ```eos.dOm_aux``` method. This method requires the dimensionless density vector, the temp_aux tuple computed with the ```eos.temperature_aux```, the dimensionless equilibrium chemical potential, and the equilibrium pressure.

In [18]:
# dimentionless density vector
rhoad = x / vlm * rhofactorm

# dimenstionless equilibrium pressure 
P = 1.01325 # bar
Pad = P*Pfactor

eosm.dOm_aux(rhoad, temp_aux, muad, Pad)

array([-9.23967584e-05])